In [29]:
results_file = '11_16-12_22-class_3_5_isoerror_results.np'

In [2]:
#!sshpass -p 'password' scp rns365@prince.hpc.nyu.edu:/home/rns365/quality_quantity_tradeoff/{results_file} /Users/rubenstern/Desktop/

In [3]:
import pickle
import numpy as np
import torch

import plotly
plotly.tools.set_credentials_file(username='rubenstern', api_key='tsHqUIBaeDLvOjJAiWMI')

import plotly.plotly as py
import plotly.graph_objs as go

In [4]:
with open(results_file, 'rb') as fd:
    data = pickle.load(fd)
    
data = np.array([[100*(1-result['quality']), result['quantity'], float(result['accuracy'])] for result in data])

In [5]:
qualities = np.sort(np.unique(data[:,0]))
quantities = np.sort(np.unique(data[:,1]))

In [6]:
qualities

array([ 60.,  65.,  70.,  75.,  80.,  85.,  90.,  95., 100.])

In [7]:
quantities

array([ 2000.,  3000.,  4000.,  5000.,  6000.,  7000.,  8000.,  9000.,
       10000.])

In [8]:
#data

In [9]:
data[58,2] = (data[57,2]+data[59,2])/2

In [13]:
def add_isocost_curves(iso_data, costs, error_rate, colors):
    iso_data2 = iso_data.copy()
    q_max = 10000
    curves_1, curves_2 = [], []
    for cost in costs:
        quanti, quali = [], []
        for i in range(3):
            if cost/float(i+1) <= q_max:
                quanti.append(cost/float(i+1))
                quali.append(100 * (1 - error_rate**(i+1)))
        curves_1.append([quanti, quali])
        
        quantis = np.arange(1,q_max)
        qualis = get_qualities(cost, error_rate, quantis)
        x = np.arange(int(cost / 2) + 1, min(cost, q_max-1)).astype(int)
        curves_2.append([x, 100 * qualis[x]])
        
    for i in range(len(costs)):
        iso_data2.append(go.Scatter(x=curves_1[i][0],
                                    y=curves_1[i][1],
                                    showlegend=False,
                                    mode = 'markers',
                                    marker = dict(size = 10, color = colors[i],
                                                 line = dict(width = 2, color = 'rgb(0, 0, 0)') )))
        iso_data2.append(go.Scatter(x=curves_2[i][0],
                                    y=curves_2[i][1],
                                    showlegend=False,
                                    mode = 'lines',
                                    line = dict(color = colors[i], width=2) ))
    return iso_data2

In [14]:
def get_qualities(cost_tot, error_rate, quantities):
        return (1-error_rate)**2 + cost_tot * error_rate * (1-error_rate) / quantities

def generate_isoerror_plot_with_isocost(data, costs, error_rate, colors):
    acc_array = []
    for q in qualities:
        selected_quality = data[data[:,0]==q]
        line_acc = selected_quality[selected_quality[:,1].argsort()][:,2]
        acc_array.append(list(line_acc))
    iso_data=[
        go.Contour(
            z=acc_array,
            x=quantities,
            y=qualities,
            contours=dict(
                coloring ='heatmap',
                showlabels = True,
                labelfont = dict(
                    family = 'Raleway',
                    size = 12,
                    color = 'white',
                )
            )
        )
    ]
    
    iso_data = add_isocost_curves(iso_data, costs, error_rate, colors)
    
    return iso_data


In [16]:
colors = ['rgba(152, 0 , 0, .8)', 'rgb(0, 0, 0)', 'rgb(255, 255, 255)']

iso_data = generate_isoerror_plot_with_isocost(data, [6000,10000,15000], 0.15, colors)

In [18]:
fig = py.iplot(iso_data)

In [32]:
import plotly.io as pio
class_list = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

def plot_save(results_file, iso_data, save_dir):
    class1 = int(results_file.split('class_')[1].split('_')[0])
    class1 = class_list[class1]
    class2 = int(results_file.split('class_')[1].split('_')[1])
    class2 = class_list[class2]
    
    layout=go.Layout(xaxis={'title':'quantity'}, yaxis={'title':'quality'})
    f = go.Figure(iso_data, layout)
    figure_file = os.path.join(save_dir, 'isocost_{}_{}.png'.format(class1, class2))
    pio.write_image(f, figure_file)
    
       
def generate_plot_save(results_file, save_dir=''):
    iso_data = get_iso(results_file)
    plot_save(results_file, iso_data, save_dir)

In [33]:
results_file

'11_16-12_22-class_3_5_isoerror_results.np'

In [34]:
import os
plot_save(results_file, iso_data, '')